In [2]:
import torch
from torchaudio.compliance.kaldi import fbank
from torch.utils.benchmark import Timer
from jiwer import wer
import os
from chunkformer_vpb.model_utils import prepare_input_file, decode_long_form, get_default_args
from chunkformer_vpb.model_utils import init, dump_module_structure, decode_aed_long_form



model_checkpoint = "../../../chunkformer-large-vie"  # adjust if needed
output_dir = "model_architect"
os.makedirs(output_dir, exist_ok=True)

# === Load model ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, char_dict = init(model_checkpoint, device)
model.eval()

##
print(char_dict)



🧾 Loaded checkpoint from: ../../../chunkformer-large-vie/pytorch_model.bin
📦 Checkpoint keys: ['encoder.global_cmvn.mean', 'encoder.global_cmvn.istd', 'encoder.embed.out.weight', 'encoder.embed.out.bias', 'encoder.embed.conv.0.weight'] ... (total 813)
🔍 AED decoder head included in checkpoint? ✅ YES
📊 Model total params: 113,852,240, trainable: 113,852,240
{0: '<blank>', 1: '<unk>', 2: '.', 3: 'a', 4: 'ab', 5: 'ac', 6: 'ace', 7: 'ach', 8: 'ack', 9: 'ad', 10: 'ada', 11: 'ade', 12: 'ado', 13: 'ael', 14: 'ag', 15: 'aga', 16: 'agan', 17: 'age', 18: 'agi', 19: 'ago', 20: 'agram', 21: 'ague', 22: 'ah', 23: 'ai', 24: 'ail', 25: 'ain', 26: 'ainer', 27: 'air', 28: 'ak', 29: 'aka', 30: 'aker', 31: 'aki', 32: 'ako', 33: 'aku', 34: 'al', 35: 'ala', 36: 'ald', 37: 'ale', 38: 'alem', 39: 'ali', 40: 'alia', 41: 'alin', 42: 'all', 43: 'alo', 44: 'aly', 45: 'am', 46: 'ama', 47: 'aman', 48: 'ame', 49: 'ami', 50: 'amin', 51: 'amoto', 52: 'an', 53: 'ana', 54: 'ance', 55: 'and', 56: 'anda', 57: 'ander', 5

In [1]:
from chunkformer_vpb.training.finetune_config import FinetuneConfig


# 1. Load config
cfg = FinetuneConfig.from_yaml("../../config/finetune_config.yaml")

# 4. Test cases
test_texts = [
    "xử lý dữ liệu",
    "quốc hội",
    "self-service",
    "đặc biệt",
    "học máy nâng cao",
]

for text in test_texts:
    ids = cfg.tokenizer.tokenizer.tokenize(text)
    toks = [cfg.tokenizer.vocab[i] for i in ids]
    print(f"\nText:      {text}")
    print(f"Token IDs: {ids}")
    print(f"Tokens:    {toks}")


Text:      xử lý dữ liệu
Token IDs: [6591, 3636, 2433, 3517]
Tokens:    ['▁xử', '▁lý', '▁dữ', '▁liệu']

Text:      quốc hội
Token IDs: [4966, 3059]
Tokens:    ['▁quốc', '▁hội']

Text:      self-service
Token IDs: [5216, 1, 740, 854, 159]
Tokens:    ['▁self', '<unk>', 'ser', 'vi', 'ce']

Text:      đặc biệt
Token IDs: [6844, 1484]
Tokens:    ['▁đặc', '▁biệt']

Text:      học máy nâng cao
Token IDs: [3037, 3958, 4541, 1755]
Tokens:    ['▁học', '▁máy', '▁nâng', '▁cao']
